Data manipulations should be implemented in SQL.

Plan management (like creating new plan periods, data approval or changing planning status) needs to be done in form of python function calls.

In [1]:
pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 487.6 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 2.6 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.2 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import psycopg2
from psycopg2.extensions import cursor


def del_plan_data(cur: cursor, year: int, quarter: int):
    delete_plan_data_query = """
    DELETE FROM plan_data
    WHERE quarterid = %s;"""
    cur.execute(delete_plan_data_query, (f"{year}.{quarter}",))


def del_plan_status(cur: cursor, year: int, quarter: int):
    delete_plan_status_query = """
    DELETE FROM plan_status
    WHERE quarterid = %s;"""
    cur.execute(delete_plan_status_query, (f"{year}.{quarter}",))


def insert_plan_dstatus(cur: cursor, year: int, quarter: int, user: str):
    insert_plan_status_query = """
    INSERT INTO plan_status 
        (quarterid, country, status, modifieddatetime, author)
    SELECT DISTINCT 
    %s AS quarterid, countrycode AS country, 'R' AS status, NOW(), %s AS author
    FROM country2;"""
    cur.execute(insert_plan_status_query, (f"{year}.{quarter}", user))


def insert_plan_data(cur: psycopg2.extensions.cursor, year: int, quarter: int):
    generate_and_copy_plan_data_query = """
    WITH avg_sales AS (
        SELECT c.countrycode AS country, cs.qr AS quarterid, cs.categoryid AS pcid, AVG(cs.salesamt) 
            FILTER (WHERE cs.ccls IN ('A', 'B')) 
            OVER (PARTITION BY c.countrycode, cs.categoryid) AS avg_sales
        FROM company_sales cs
        JOIN company c ON cs.cid = c.id
        -- JOIN plan_status ps ON ps.quarterid = cs.qr AND ps.country = c.countrycode
        -- WHERE quarterid = %s
    )
    INSERT INTO plan_data (versionid, country, quarterid, pcid, salesamt)
    SELECT 'N' AS versionid, country, %s AS quarterid, pcid, COALESCE(avg_sales, 0)
    FROM avg_sales
    UNION ALL
    SELECT 'P' AS versionid, country, %s AS quarterid, pcid, COALESCE(avg_sales, 0)
    FROM avg_sales
    ON CONFLICT (quarterid, country, pcid, versionid) DO NOTHING;"""
    # SELECT * FROM avg_sales;"""
    cur.execute(
        generate_and_copy_plan_data_query,
        (f"{year}.{quarter}", f"{year}.{quarter}", f"{year}.{quarter}"),
    )


def start_planning(year, quarter, user, pwd):
    # Establish a connection to the database
    C = "postgres"
    conn = psycopg2.connect(dbname=C, user=user, password=pwd, host="localhost")

    # Create a cursor object
    cur = conn.cursor()

    try:
        del_plan_data(cur, year, quarter)
    except Exception as e:
        conn.commit()
        print(f"del_plan_data failed: {e}")
        return

    try:
        del_plan_status(cur, year, quarter)
    except Exception as e:
        conn.commit()
        print(f"del_plan_status failed: {e}")
        return

    try:
        insert_plan_dstatus(cur, year, quarter, user)
    except Exception as e:
        print(f"insert_plan_dstatus failed: {e}")
        return

    try:
        insert_plan_data(cur, year, quarter)
    except Exception as e:
        print(f"insert_plan_data failed: {e}")
        return

    try:
        # Commit the transaction
        conn.commit()
        print("Planning data successfully initialized.")

    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()

    finally:
        # Close the cursor and connection
        cur.close()
        conn.close()


start_planning(2014, 1, "ivan", "password_for_ivan")

Planning data successfully initialized.


In [1]:
import psycopg2

C = "postgres"


def set_lock(year: int, quarter: int, user: str, pwd: str):
    conn = psycopg2.connect(dbname=C, user=user, password=pwd, host="localhost")
    cur = conn.cursor()

    try:
        # Update plan_status to set status from R to L for records associated with the target quarter and year
        # and connected to the current user in the country_managers table
        update_query = """
        UPDATE plan_status
        SET status = 'L', modifieddatetime = CURRENT_TIMESTAMP, author = %s
        WHERE quarterid = %s AND country IN (
            SELECT country FROM country_managers WHERE username = %s
        ) AND status = 'R';
        """
        cur.execute(update_query, (user, f"{year}.{quarter}", user))

        conn.commit()
        print("Lock set successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()


def remove_lock(year: int, quarter: int, user: str, pwd: str):
    conn = psycopg2.connect(dbname=C, user=user, password=pwd, host="localhost")
    cur = conn.cursor()

    try:
        # Update plan_status to set status from L to R for records associated with the target quarter and year
        # and connected to the current user in the country_managers table
        update_query = """
        UPDATE plan_status
        SET status = 'R', modifieddatetime = CURRENT_TIMESTAMP, author = %s
        WHERE quarterid = %s AND country IN (
            SELECT country FROM country_managers WHERE username = %s
        ) AND status = 'L' AND author = %s;
        """
        cur.execute(update_query, (user, f"{year}.{quarter}", user, user))

        conn.commit()
        print("Lock removed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()


# Example usage of the functions
set_lock(2014, 1, "kirill", "password_for_kirill")
set_lock(2014, 1, "sophie", "password_for_sophie")

Lock set successfully.
Lock set successfully.


In [2]:
remove_lock(2014, 1, "kirill", "password_for_kirill")
remove_lock(2014, 1, "sophie", "password_for_sophie")

Lock removed successfully.
Lock removed successfully.


In [3]:
import psycopg2


def accept_plan(year: int, quarter: int, user: str, pwd: str):
    conn = psycopg2.connect(dbname=C, user=user, password=pwd, host="localhost")
    cur = conn.cursor()

    try:
        # Define the target quarter
        target_quarter = f"{year}.{quarter}"

        # Step 1: Clear the A version of plan data for specific quarter and countries accessible to the current user
        clear_a_version_query = """
        DELETE FROM plan_data
        WHERE quarterid = %s AND versionid = 'A' AND country IN (
            SELECT country FROM country_managers WHERE username = %s
        );
        """
        cur.execute(clear_a_version_query, (target_quarter, user))

        # Step 2: Read data available to the current user from version P and save its copy as version A
        copy_to_a_version_query = """
        INSERT INTO plan_data (versionid, country, quarterid, pcid, salesamt)
        SELECT 'A' AS versionid, country, quarterid, pcid, salesamt
        FROM plan_data
        WHERE quarterid = %s AND versionid = 'P' AND country IN (
            SELECT country FROM country_managers WHERE username = %s
        );
        """
        cur.execute(copy_to_a_version_query, (target_quarter, user))

        # Step 3: Change the status of the processed from 'R' to 'A' in plan_status
        update_status_query = """
        UPDATE plan_status
        SET status = 'A', modifieddatetime = CURRENT_TIMESTAMP, author = %s
        WHERE quarterid = %s AND country IN (
            SELECT country FROM country_managers WHERE username = %s
        ) AND status = 'R';
        """
        cur.execute(update_status_query, (user, target_quarter, user))

        conn.commit()
        print("Plan accepted successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()


# Example usage of the function
accept_plan(2014, 1, "ivan", "password_for_ivan")

Plan accepted successfully.


In [4]:
accept_plan(2014, 1, "sophie", "password_for_sophie")


Plan accepted successfully.
